# Cluster Computing 

Physics and astronomy often invove large datasets or a large number of computations or both, for which simply running code on your personal machines will not suffice. 
You could need more firepower (eg. more cores), more memory or just different processors (eg. GPUs). This is where computer clusters come in. You take your code, stick it on a cluster and submit jobs to be run on the cluster remotely. Here, we'll go over the basics of how to do that. 

## Connecting to a cluster

In [Section 2](https://github.com/jeffiuliano/Penn-Summer-Computing-Training/blob/main/2_ssh_and_scp/SSH_SCP_Workbook.ipynb), we learnt to securely interact with a non-local machine. To connect to most clusters, we will use SSH as described there. For example, for those of you using *marmalade*, you will do 

Note that to connect to *marmalade*, you must be on a Penn secured network (eg. AirPennNet wi-fi) or VPN into Penn. For using the Penn VPN, you need a PennKey (UPenn username and account) and associated account setup. For details of how to VPN into Penn, go [here](https://www.isc.upenn.edu/how-to/university-vpn-getting-started-guide). 